In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.layers import UpSampling2D
from keras.datasets import cifar10

In [ ]:
import tensorflow as tf

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
train_images = UpSampling2D(size=(3,3))(x_train)

In [ ]:
test_images = UpSampling2D(size=(3,3))(x_test)

In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
train_images = np.array(train_images)

In [ ]:
test_images = np.array(test_images)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import MaxPooling2D

In [ ]:
import cv2

In [ ]:
def one_hot_encode (label):
    label = np_utils.to_categorical(label)
    return label

In [ ]:
labels_train = one_hot_encode(y_train)
labels_test = one_hot_encode(y_test)

In [ ]:
print(labels_train.shape)
print(labels_test.shape)

## Splitting

In [ ]:
def splitColor (dataset, rows, columns):
    
    split_datasets = [ [] for i in range( rows * columns ) ]
    
    rows = int (96 / rows)
    columns = int (96 / columns)
    
    for data in dataset:
        data = data.reshape( 96//rows, rows,-1, columns, 3).swapaxes(1,2).reshape(-1, rows, columns, 3)
        
        for i in range(len(split_datasets)):
            split_datasets[i].append(data[i])
    
    return np.array(split_datasets)

In [ ]:
def display(image, rows, columns):

    fig = plt.figure(figsize = (5,5))

    for i in range( rows * columns):

        fig.add_subplot(rows, columns, i + 1)

        plt.imshow(image[i], cmap = "gray")
        plt.axis('off')
    plt.show()

In [ ]:
split_datasets_train4 = splitColor(train_images, 2, 2)
split_datasets_test4 = splitColor(test_images, 2, 2)

In [ ]:
split_datasets_train9 = splitColor(train_images, 3, 3)
split_datasets_test9 = splitColor(test_images, 3, 3)

In [ ]:
split_datasets_train6 = splitColor(train_images, 3, 2)
split_datasets_test6 = splitColor(test_images, 3, 2)

In [ ]:
print(split_datasets_train6.shape)

## Model

In [ ]:
def combine(predictions):
    p = []
    for j in range(len(predictions[0])):
        y = np.array([np.argmax(predictions[i][j]) for i in range(len(predictions))])
        p.append(np.bincount(y).argmax())
    return np_utils.to_categorical(p)

### VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

In [ ]:
net4 = []
# load model without classifier layers
for i in range(4):
    net4.append(VGG16(include_top=False, input_shape=(48, 48, 3)))

for i in range(4):
    for layer in net4[i].layers:
        layer.trainable = False

for i in range(4):
    # add new classifier layers
    flat1 = Flatten()(net4[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(10, activation='softmax')(class1)
    # define new model
    net4[i] = Model(inputs=net4[i].inputs, outputs=output)

net4[0].summary()

In [ ]:
for i in range(4):
    net4[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net4[i].fit(split_datasets_train4[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
net9 = []
# load model without classifier layers
for i in range(9):
    net9.append(VGG16(include_top=False, input_shape=(32, 32, 3)))

for i in range(9):
    for layer in net9[i].layers:
        layer.trainable = False
        
for i in range(9):
    # add new classifier layers
    flat1 = Flatten()(net9[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    net9[i] = Model(inputs=net9[i].inputs, outputs=output)
    
net9[0].summary()

In [ ]:
for i in range(9):
    net9[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net9[i].fit(split_datasets_train9[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
net6 = []
# load model without classifier layers
for i in range(6):
    net6.append(VGG16(include_top=False, input_shape=(32, 32, 3)))

for i in range(6):
    for layer in net6[i].layers:
        layer.trainable = False
        
for i in range(6):
    # add new classifier layers
    flat1 = Flatten()(net6[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    net6[i] = Model(inputs=net6[i].inputs, outputs=output)
    
net6[0].summary()

In [ ]:
for i in range(6):
    net6[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net6[i].fit(split_datasets_train6[i], labels_train, epochs=10, batch_size = 128,verbose=2)

### ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

In [ ]:
Rnet4 = []
# load model without classifier layers
for i in range(4):
    Rnet4.append(ResNet50(include_top=False, input_shape=(48, 48, 3)))

for i in range(4):
    for layer in Rnet4[i].layers:
        layer.trainable = False

for i in range(4):
    # add new classifier layers
    flat1 = Flatten()(Rnet4[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(10, activation='softmax')(class1)
    # define new model
    Rnet4[i] = Model(inputs=Rnet4[i].inputs, outputs=output)

In [ ]:
for i in range(4):
    Rnet4[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    Rnet4[i].fit(split_datasets_train4[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
Rnet9 = []
# load model without classifier layers
for i in range(9):
    Rnet9.append(ResNet50(include_top=False, input_shape=(32, 32, 3)))

for i in range(9):
    for layer in Rnet9[i].layers:
        layer.trainable = False

for i in range(9):
    # add new classifier layers
    flat1 = Flatten()(Rnet9[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    Rnet9[i] = Model(inputs=Rnet9[i].inputs, outputs=output)
    # summarize
Rnet9[0].summary()

In [ ]:
for i in range(9):
    Rnet9[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    Rnet9[i].fit(split_datasets_train9[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
Rnet6 = []
# load model without classifier layers
for i in range(6):
    Rnet6.append(ResNet50(include_top=False, input_shape=(32, 48, 3)))

for i in range(6):
    for layer in Rnet6[i].layers:
        layer.trainable = False

for i in range(6):
    # add new classifier layers
    flat1 = Flatten()(Rnet6[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    Rnet6[i] = Model(inputs=Rnet6[i].inputs, outputs=output)
    # summarize
Rnet6[0].summary()

In [ ]:
for i in range(6):
    Rnet6[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    Rnet6[i].fit(split_datasets_train6[i], labels_train, epochs=10, batch_size = 128,verbose=2)

## Predcitions

### VGG16

In [ ]:
vgg16pred4 = [ [] for i in range(len(net4))]

for i in range(len(net4)):
    vgg16pred4[i] = net4[i].predict(split_datasets_test4[i])

In [ ]:
vgg16pred9 = [ [] for i in range(len(net9))]

for i in range(len(net9)):
    vgg16pred9[i] = net9[i].predict(split_datasets_test9[i])

In [ ]:
vgg16pred6 = [ [] for i in range(len(net6))]

for i in range(len(net6)):
    vgg16pred6[i] = net6[i].predict(split_datasets_test6[i])

### ResNet50

In [ ]:
ResNet50pred4 = [ [] for i in range(len(Rnet4))]

for i in range(len(Rnet4)):
    ResNet50pred4[i] = Rnet4[i].predict(split_datasets_test4[i])

In [ ]:
ResNet50pred9 = [ [] for i in range(len(Rnet9))]

for i in range(len(Rnet9)):
    ResNet50pred9[i] = Rnet9[i].predict(split_datasets_test9[i])

In [ ]:
ResNet50pred6 = [ [] for i in range(len(Rnet6))]

for i in range(len(Rnet6)):
    ResNet50pred6[i] = Rnet6[i].predict(split_datasets_test6[i])

## Combine

### VGG16

In [ ]:
vgg16p4 = combine(vgg16pred4)

In [ ]:
vgg16p9 = combine(vgg16pred9)

In [ ]:
vgg16p6 = combine(vgg16pred6)

### ResNet50

In [ ]:
ResNet50p4 = combine(ResNet50pred4)

In [ ]:
ResNet50p9 = combine(ResNet50pred9)

In [ ]:
ResNet50p6 = combine(ResNet50pred6)

## Accuracy

### VGG16

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p6)
metric.result().numpy()

### ResNet50

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p6)
metric.result().numpy()

# Single

### VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(75, 75, 3))

for layer in model.layers:
        layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(6, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_train, labels_train,validation_data=(data_test, labels_test), epochs=10, batch_size = 128,verbose=2)

### ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input
# load model
model = ResNet50()
# summarize the model
input_tensor = Input(shape=(75, 75, 3))

model = ResNet50(input_tensor=input_tensor,include_top=False)

for layer in model.layers:
        layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(6, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_train, labels_train,validation_data=(data_test, labels_test), epochs=10, batch_size = 128,verbose=2)

## Convolutions

In [ ]:
class ModelConv:
    def __init__(self, n_classes):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3,3),input_shape = (48, 48, 3), padding='same',  activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D((2, 2)))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(Flatten())
        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Cnet4 = []

for i in range(4):
    Cnet4.append( ModelConv(10) )
    
Cnet4[0].model.summary()

In [ ]:
for i in range(len(Cnet4)):
    Cnet4[i].model.fit(split_datasets_train4[i], labels_train,epochs=10, batch_size=128, verbose=2)

### Predictions

In [ ]:
Cpred4 = [ [] for i in range(len(Cnet4))]

for i in range(len(Cnet4)):
    Cpred4[i] = Cnet4[i].model.predict(split_datasets_test4[i])

In [ ]:
Cpred9 = [ [] for i in range(len(Cnet9))]

for i in range(len(Cnet9)):
    Cpred9[i] = Cnet9[i].model.predict(split_datasets_test9[i])

In [ ]:
Cpred6 = [ [] for i in range(len(Cnet6))]

for i in range(len(Cnet6)):
    Cpred6[i] = Cnet6[i].model.predict(split_datasets_test6[i])

### Combine

In [ ]:
Cp4 = combine(Cpred4)

In [ ]:
Cp9 = combine(Cpred9)

In [ ]:
Cp6 = combine(Cpred6)

### Accuracy

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Cp4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Cp9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Cp6)
metric.result().numpy()

## Networks

### Single

In [ ]:
class ModelOne:
    def __init__(self, n_classes):
        self.model = Sequential()
        self.model.add(Flatten())
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Snet4 = []

for i in range(4):
    Snet4.append( ModelOne(10) )
    
for i in range(len(Snet4)):
    Snet4[i].model.fit(split_datasets_train4[i], labels_train,epochs=10, batch_size=128, verbose=2)

In [ ]:
Snet4[0].model.summary()

In [ ]:
Snet9 = []

for i in range(9):
    Snet9.append( ModelOne(10) )
    
for i in range(len(Snet9)):
    Snet9[i].model.fit(split_datasets_train9[i], labels_train,epochs=10, batch_size=128, verbose=2)

In [ ]:
Snet6 = []

for i in range(6):
    Snet6.append( ModelOne(10) )
    
for i in range(len(Snet6)):
    Snet6[i].model.fit(split_datasets_train6[i], labels_train,epochs=10, batch_size=128, verbose=2)

### Prediction

In [ ]:
Spred4 = [ [] for i in range(len(Snet4))]

for i in range(len(Snet4)):
    Spred4[i] = Snet4[i].model.predict(split_datasets_test4[i])

In [ ]:
Spred9 = [ [] for i in range(len(Snet9))]

for i in range(len(Snet9)):
    Spred9[i] = Snet9[i].model.predict(split_datasets_test9[i])

In [ ]:
Spred6 = [ [] for i in range(len(Snet6))]

for i in range(len(Snet6)):
    Spred6[i] = Snet6[i].model.predict(split_datasets_test6[i])

### Combine

In [ ]:
def Scombine (predictions):
    p = []
    for j in range(len(predictions[0])):
        y = np.array([np.argmax(predictions[i][j]) for i in range(len(predictions))])
        p.append(np.bincount(y).argmax())
    return np_utils.to_categorical(p)

In [ ]:
Sp4 = combine(Spred4)

In [ ]:
Sp4.shape

In [ ]:
Sp9 = combine(Spred9)

In [ ]:
Sp6 = combine(Spred6)

### Accuracy

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Sp4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Sp9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Sp6)
metric.result().numpy()

### Multiple

In [ ]:
class ModelMult:
    def __init__(self, n_classes):
        self.model = Sequential()
        self.model.add(Flatten())
        self.model.add(Dense(2500, activation='relu'))
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Mnet4 = []

for i in range(4):
    Mnet4.append( ModelMult(10) )
    
for i in range(len(Mnet4)):
    Mnet4[i].model.fit(split_datasets_train4[i], labels_train,epochs=10, batch_size=128, verbose=2)

In [ ]:
Mnet9 = []

for i in range(9):
    Mnet9.append( ModelOne(10) )
    
for i in range(len(Mnet9)):
    Mnet9[i].model.fit(split_datasets_train9[i], labels_train,epochs=10, batch_size=128, verbose=2)

In [ ]:
Mnet6 = []

for i in range(6):
    Mnet6.append( ModelOne(10) )
    
for i in range(len(Mnet6)):
    Mnet6[i].model.fit(split_datasets_train6[i], labels_train,epochs=10, batch_size=128, verbose=2)

### Prediction

In [ ]:
Mpred4 = [ [] for i in range(len(Mnet4))]

for i in range(len(Mnet4)):
    Mpred4[i] = Mnet4[i].model.predict(split_datasets_test4[i])

In [ ]:
Spred9 = [ [] for i in range(len(Snet9))]

for i in range(len(Snet9)):
    Spred9[i] = Snet9[i].model.predict(split_datasets_test6[i])

In [ ]:
Spred6 = [ [] for i in range(len(Snet6))]

for i in range(len(Snet6)):
    Spred6[i] = Snet6[i].model.predict(split_datasets_test6[i])

### Combine

In [ ]:
Mp4 = combine(Mpred4)

In [ ]:
Mp9 = combine(Mpred9)

In [ ]:
Mp6 = combine(Mpred6)

### Accuracy

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Mp4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Mp9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,Mp6)
metric.result().numpy()